# Feature Engineering

## Import libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing # puts the features in standard format
from sklearn import pipeline 

## Import Dataframe

In [3]:
df = pd.read_csv('csv_files/12_12_DF_log_state_FINAL.csv')

### Initial look at dataframe

In [8]:
df.tail()

,Life Expectancy,Percent Smokers,Percent Obese,Unemployment_rate_2010,log_food_stamps,Alaska,Arizona,Arkansas,California,Colorado,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Min Wages Dummies
2978,0.102956,0.306336,-0.751646,-0.877782,-0.773864,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2979,2.342667,-2.827030,-4.134520,-0.530616,-0.509604,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2980,-0.084430,-0.398064,-0.183891,-0.751540,-5.517516,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2981,0.477728,-0.422354,-1.532309,-1.098706,1.532639,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2982,0.419728,0.063440,0.170956,-1.445873,-0.972225,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Test, Train, Split

In [17]:
# the target is panda series of the dependent variable
target = df['Life Expectancy'] 

# make dataframe of just feature, or independent variables
features = df.drop(columns=['Life Expectancy']) # taking target out of dataframe

features.tail()

,Percent Smokers,Percent Obese,Unemployment_rate_2010,log_food_stamps,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Min Wages Dummies
2978,0.306336,-0.751646,-0.877782,-0.773864,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2979,-2.827030,-4.134520,-0.530616,-0.509604,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2980,-0.398064,-0.183891,-0.751540,-5.517516,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2981,-0.422354,-1.532309,-1.098706,1.532639,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2982,0.063440,0.170956,-1.445873,-0.972225,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [22]:
# split the data to train and test our model
X_train, X_test, Y_train, Y_test = train_test_split(features, target, random_state=32,test_size=0.2)


In [37]:
# check the length of our train and test dataframes

print(' Length of ',
    '\n Feautures Training Data:',len(X_train),
    '\n Features Test Data:', len(X_test), 
    '\n Target Test Data:', len(Y_train),
    '\n Target Test Data:', len(Y_test))

 Length of  
 Feautures Training Data: 2386 
 Features Test Data: 597 
 Target Test Data: 2386 
 Target Test Data: 597


## Creating Interact and Polynomial Terms

# Feature Selection

## Using Filter Methods

## Applying a Wrapper Method to Filtered features

# Final Thoughts